In [2]:
!nvidia-smi

Sat May 21 01:31:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -q transformers
!pip install -q datasets

     |████████████████████████████████| 4.2 MB 7.1 MB/s 
     |████████████████████████████████| 6.6 MB 34.1 MB/s 
     |████████████████████████████████| 596 kB 56.2 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 346 kB 9.8 MB/s 
     |████████████████████████████████| 212 kB 48.3 MB/s 
     |████████████████████████████████| 140 kB 54.2 MB/s 
     |████████████████████████████████| 1.1 MB 49.4 MB/s 
     |████████████████████████████████| 127 kB 63.7 MB/s 
     |████████████████████████████████| 271 kB 61.3 MB/s 
     |████████████████████████████████| 94 kB 2.5 MB/s 
     |████████████████████████████████| 144 kB 55.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
#transformers
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler 
import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments

In [5]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_1", num_labels=3)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Some weights of the model checkpoint at /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_1 were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_1 and are newly initialized: ['classifi

In [6]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

# Load Dataset


In [7]:
train_dataset_path = "/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/aug_clean_train.csv"
test_dataset_path = "/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/clean_test.csv"

In [8]:
from datasets import Dataset, load_dataset

dataset = load_dataset('csv', data_files={'train': train_dataset_path,
                                          'val': test_dataset_path})

Using custom data configuration default-c23d9ec77aab9cc0


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c23d9ec77aab9cc0/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

## Preprocess data

In [9]:
dataset = dataset.rename_column("text_clean", "texts")
dataset = dataset.rename_column("Sentiment", "labels")
# dataset['train'], dataset['validation'] = dataset['train'].train_test_split(0.2).values()

In [10]:
def label_mapping(examples):
    label_to_idx = {'Extremely Negative': 0, 'Negative': 0, 'Neutral': 1,
                    'Positive': 2, 'Extremely Positive': 2}
    examples["labels"] = label_to_idx[examples["labels"]]
    return examples
    
dataset = dataset.map(label_mapping)

  0%|          | 0/48483 [00:00<?, ?ex/s]

  0%|          | 0/3787 [00:00<?, ?ex/s]

In [11]:
def tokenizer_function(examples):
    result = tokenizer(examples["texts"], max_length=128,
                       padding='max_length', truncation=True)
    result["labels"] = examples["labels"]
    return result

tokenized_datasets = dataset.map(tokenizer_function, batched=True, remove_columns=['texts', 'labels'])

  0%|          | 0/49 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 48483
    })
    val: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 3787
    })
})

In [13]:
tokenized_datasets['train'][0]['labels']

1

# Fine-tuning Roberta with Trainer API

In [14]:
from datasets import load_metric

def compute_metrics(eval_preds):
    accuracy_metric = load_metric("accuracy")
    precision_metric = load_metric("precision")
    recall_metric = load_metric("recall")
    f1_metric = load_metric("f1")

    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average="macro")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="macro")
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")

    return {
        "accuracy": accuracy['accuracy'],
        "precision": precision['precision'],
        "recall": recall['recall'],
        "f1": f1['f1'],
    }

In [15]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    "/content/output",
    num_train_epochs=20,
    evaluation_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model = 'f1',
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    save_strategy = "epoch",
    logging_steps=200
)

In [16]:
# trainer=Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets['train'],
#     eval_dataset=tokenized_datasets['val'],
#     compute_metrics=compute_metrics,
# )

In [17]:
# trainer.train()

# Hyperparameter search

In [18]:
! pip install -q optuna
! pip install -q ray[tune]

     |████████████████████████████████| 308 kB 7.7 MB/s 
     |████████████████████████████████| 210 kB 61.6 MB/s 
     |████████████████████████████████| 81 kB 7.9 MB/s 
     |████████████████████████████████| 78 kB 6.4 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 112 kB 63.7 MB/s 
     |████████████████████████████████| 146 kB 64.5 MB/s 
     |████████████████████████████████| 53.2 MB 1.2 MB/s 
     |████████████████████████████████| 4.1 MB 42.9 MB/s 
     |████████████████████████████████| 8.8 MB 50.9 MB/s 
     |████████████████████████████████| 125 kB 54.4 MB/s 
     |████████████████████████████████| 461 kB 56.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [19]:
def model_init():
    return RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_1", num_labels=3)

In [20]:
trainer=Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    compute_metrics=compute_metrics,
)

loading configuration file /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_1/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file /content/drive/M

In [21]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-5, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 3, 7),
    }

best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize", hp_space=my_hp_space)

[I 2022-05-21 01:32:56,387] A new study created in memory with name: no-name-9609b58f-59d8-480c-b083-b7e07ab39a11
Trial:
loading configuration file /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_1/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_versio

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.832700,0.750909,0.676789,0.646650,0.652148,0.647383
2,0.669100,0.662723,0.731186,0.711971,0.700017,0.703782
3,0.610300,0.614052,0.753895,0.742549,0.720388,0.727998
4,0.564900,0.574222,0.771059,0.757911,0.742347,0.748955
5,0.532100,0.565821,0.779773,0.767976,0.749528,0.757521
6,0.516600,0.558118,0.780565,0.773071,0.746545,0.757245
7,0.500500,0.556641,0.780565,0.769145,0.750627,0.758512


***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64


Saving model checkpoint to /content/output/run-0/checkpoint-758
Configuration saved in /content/output/run-0/checkpoint-758/config.json
Model weights saved in /content/output/run-0/checkpoint-758/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-0/checkpoint-1516
Configuration saved in /content/output/run-0/checkpoint-1516/config.json
Model weights saved in /content/output/run-0/checkpoint-1516/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-0/checkpoint-2274
Configuration saved in /content/output/run-0/checkpoint-2274/config.json
Model weights saved in /content/output/run-0/checkpoint-2274/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-0/checkpoint-3032
Configuration saved in /content/output/run-0/checkpoint-3032/config.json
M

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.526700,0.474627,0.818590,0.815323,0.789676,0.799963
2,0.377300,0.421692,0.847901,0.848334,0.820767,0.832008
3,0.328600,0.397975,0.862424,0.857621,0.836158,0.845591


...

***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-7/checkpoint-758
Configuration saved in /content/output/run-7/checkpoint-758/config.json
Model weights saved in /content/output/run-7/checkpoint-758/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-7/checkpoint-1516
Configuration saved in /content/output/run-7/checkpoint-1516/config.json
Model weights saved in /content/output/run-7/checkpoint-1516/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-7/checkpoint-2274
Configuration saved in /content/output/run-7/checkpoint-2274/config.json
Model weights saved in /content/output/run-7/checkpoint-2274/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/output/run-7/checkpo

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.779900,0.706958,0.698178,0.670449,0.668034,0.668372
2,0.630700,0.632496,0.749142,0.733399,0.716817,0.722967


***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-8/checkpoint-758
Configuration saved in /content/output/run-8/checkpoint-758/config.json
Model weights saved in /content/output/run-8/checkpoint-758/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
[I 2022-05-21 08:19:28,031] Trial 8 pruned. 
Trial:
loading configuration file /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_1/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABE

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.401100,0.390509,0.863480,0.863000,0.841036,0.850573
2,0.280800,0.318483,0.890943,0.888099,0.873840,0.880289
3,0.202300,0.344758,0.896752,0.902217,0.871351,0.884461
4,0.137300,0.352822,0.895696,0.887245,0.869709,0.877616
5,0.095400,0.414347,0.893583,0.880282,0.873507,0.876647


***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-9/checkpoint-758
Configuration saved in /content/output/run-9/checkpoint-758/config.json
Model weights saved in /content/output/run-9/checkpoint-758/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-9/checkpoint-1516
Configuration saved in /content/output/run-9/checkpoint-1516/config.json
Model weights saved in /content/output/run-9/checkpoint-1516/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-9/checkpoint-2274
Configuration saved in /content/output/run-9/checkpoint-2274/config.json
Model weights saved in /content/output/run-9/checkpoint-2274/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-9/checkpoint-3032
Con